# TriMet Data Sampling Methods

Kenneth Liao
<br>
5/21/2018



This document demonstrates the sampling methods employed on the TriMet vehicle data. Here I only show how the one month data was sampled. The other time frames were generated following a similar procedure. Raw data was downloaded from https://viewer.db4iot.com/trimet in monthly increments spanning September 2016 to July 2017, for a total of 11 months of data. Data for one hour, day, and week time frames was taken from January 2017.

---

---

In [1]:
import pandas as pd

We start by importing the raw data for January 2017. A quick summary shows there are 32 total columns, 26.9 million rows, and 5.9GB of data. This is for a single month. If we were to pull data for all of 2017, we would be working with 72GB of data!! We have to reduce this file size significantly, especially to aggregate data over longer periods of time. One efficient way to manage very large data sets like this is to break the single data table into smaller tables that share a common theme. This is the idea behind relational databases. We will employ this technique for managing our very large data set. 

In [2]:
# Read in the raw data for Jan 2017
data = pd.read_csv('export_2018-05-04T03_35_05.863Z.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26894006 entries, 0 to 26894005
Data columns (total 32 columns):
bearing                         int64
blockID                         int64
city                            object
delay                           int64
device_id                       int64
direction                       int64
event_timestamp                 int64
expires                         object
garage                          object
inCongestion                    bool
lastLocID                       int64
lastStopSeq                     int64
loadPercentage                  int64
locationInScheduleDay           int64
messageCode                     int64
newTrip                         bool
nextLocID                       int64
nextStopSeq                     int64
offRoute                        bool
routeNumber                     int64
serviceDate                     object
signMessage                     object
signMessageLong                 object
state   

In [3]:
# Convert 'event_timestamp' column into pandas datetime object
data['event_timestamp'] = pd.to_datetime(data['event_timestamp'], unit='s')
data.head()

,bearing,blockID,city,delay,device_id,direction,event_timestamp,expires,garage,inCongestion,...,signMessageLong,state,system,tripID,type,vehicle_id,vehicle_location_latitude,vehicle_location_longitude,vehicle_location_latitude.1,vehicle_location_longitude.1
0,147,9066,portland,-31,317,1,2017-01-01 08:00:00,2017-01-01T08:08:31,RUBY,False,...,MAX Orange Line to Milwaukie,oregon,trimet,7004120.0,rail,317,45.483499,-122.643220,45.483499,-122.643220
1,90,2037,portland,-294,2285,1,2017-01-01 08:00:02,2017-01-01T08:04:00,POWELL,False,...,20 Burnside/Stark to Gresham TC,oregon,trimet,6986803.0,bus,2285,45.519149,-122.491014,45.519149,-122.491014
2,0,3301,portland,0,2706,0,2017-01-01 08:00:02,2017-01-01T08:04:01,NaN,False,...,NaN,oregon,trimet,NaN,bus,2706,45.444867,-122.640800,45.444867,-122.640800
3,20,9069,portland,-113,413,0,2017-01-01 08:00:04,2017-01-01T08:07:53,RUBY,False,...,MAX Orange Line to City Center/Expo Center,oregon,trimet,7003700.0,rail,413,45.439829,-122.640634,45.439829,-122.640634
4,98,3502,portland,-81,2608,0,2017-01-01 08:00:05,2017-01-01T08:04:02,CENTER,False,...,35 Macadam to Oregon City TC,oregon,trimet,6988607.0,bus,2608,45.419220,-122.663502,45.419220,-122.663502


This series will focus only on rail vehicles to limit the total data size and narrow the scope of our project. Larger data sets which include busses may be offered optionally.

In [4]:
# Ensure that we're only using data from January 2017
data = data.loc[data['event_timestamp'].dt.month==1, :]

# Limit the data to the rail vehicle type
rail_data = data.loc[data['type']=='rail', :]

# Convert tripID column to int
rail_data['tripID'] = rail_data['tripID'].astype(int)

rail_data.nunique()

C:\Users\Kenny\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


bearing                             360
blockID                              69
city                                  1
delay                              1358
device_id                           140
direction                             2
event_timestamp                 1191366
expires                         1069775
garage                                2
inCongestion                          1
lastLocID                           172
lastStopSeq                          47
loadPercentage                        1
locationInScheduleDay             78703
messageCode                          21
newTrip                               1
nextLocID                           172
nextStopSeq                          47
offRoute                              1
routeNumber                           5
serviceDate                          32
signMessage                          21
signMessageLong                      21
state                                 1
system                                1


A quick check of the number of unique values per column shows us several columns which only ever have one value. We can pull this information out and save it as meta data that applies to all entries (rows). We can then delete these columns from the main data set to reduce the final size.

In [5]:
meta= rail_data.loc[:,rail_data.nunique()==1].drop_duplicates().reset_index(drop=True)
meta

,city,inCongestion,loadPercentage,newTrip,offRoute,state,system,type
0,portland,False,0,False,False,oregon,trimet,rail


Next, we can create a dictionary file for vehicle sign messages using TriMet's convenient messageCode column. This will reduce the file size significantly since we don't have to save all of the message text in the main data table. This is our first relational table. The signMessage for any event in the main table can be obtained by referencing the corresponding messageCode in this table.

In [6]:
messages = rail_data[['messageCode', 'signMessageLong']].drop_duplicates().reset_index(drop=True)
messages

,messageCode,signMessageLong
0,968,MAX Orange Line to Milwaukie
1,970,MAX Orange Line to City Center/Expo Center
2,892,MAX Blue Line to Gresham
3,972,MAX Yellow Line to Expo Center
4,978,MAX Green Line to City Center/PSU
5,929,MAX Red Line to City Center & Beaverton
6,891,MAX Blue Line to Hillsboro
7,969,MAX Orange Line to City Center/PSU
8,971,MAX Yellow Line to City Center/Milwaukie
9,975,MAX Green Line to Clackamas Town Center


Now we can delete columns that won't be useful to our analysis, including the meta data columns and the signMessage columns.

In [7]:
# Remove unnecessary columns
events_month = rail_data.drop(['city',
               'device_id',  
               'expires',
               'inCongestion',
               'loadPercentage',
               'locationInScheduleDay',
               'newTrip',
               'offRoute', 
               'serviceDate',
               'signMessage',
               'signMessageLong',
               'state',
               'system',
               'type',
               'vehicle_location_latitude.1', 
               'vehicle_location_longitude.1'], axis=1)

events_month.reset_index(inplace=True, drop=True)

events_month.sort_values('event_timestamp')

events_month.head()

,bearing,blockID,delay,direction,event_timestamp,garage,lastLocID,lastStopSeq,messageCode,nextLocID,nextStopSeq,routeNumber,tripID,vehicle_id,vehicle_location_latitude,vehicle_location_longitude
0,147,9066,-31,1,2017-01-01 08:00:00,RUBY,13715,13,968,13716,14,290,7004120,317,45.483499,-122.643220
1,20,9069,-113,0,2017-01-01 08:00:04,RUBY,13720,1,970,13721,2,290,7003700,413,45.439829,-122.640634
2,91,9001,-17,0,2017-01-01 08:00:22,RUBY,9848,1,892,9846,2,100,6998908,110,45.521351,-122.986448
3,358,9067,-56,0,2017-01-01 08:00:22,RUBY,9299,6,972,7763,7,190,7000472,420,45.526400,-122.676568
4,358,9067,-56,0,2017-01-01 08:00:22,RUBY,9299,6,972,7763,7,190,7000472,412,45.526400,-122.676568


It seems natural for each tripID to have associated unique values from other columns. It wouldn't make sense for example for a tripID to have multiple messageCodes or routeNumbers. We can check exactly which other columns have unique values for each unique tripID by using the groupby function.

In [8]:
events_month.groupby('tripID').nunique().nunique()

bearing                        140
blockID                          1
delay                          483
direction                        1
event_timestamp               1213
garage                           1
lastLocID                       46
lastStopSeq                     47
messageCode                      1
nextLocID                       47
nextStopSeq                     47
routeNumber                      1
tripID                           1
vehicle_id                      20
vehicle_location_latitude      551
vehicle_location_longitude     565
dtype: int64

The cell above shows that 5 other columns have a 1-to-1 correspondence with tripID. Trips will thus form our 2nd relational table. The final and main table will be everything that's left: the events table.

In [9]:
trips_month = events_month[['tripID', 'vehicle_id', 'blockID', 'direction', 'garage', 'messageCode', 'routeNumber']]

# Drop duplicates and reset index
trips_month = trips_month.drop_duplicates()

# order by tripID
trips_month = trips_month.sort_values('tripID').reset_index(drop=True)

trips_month.head()

,tripID,vehicle_id,blockID,direction,garage,messageCode,routeNumber
0,6997573,408,9045,1,ELMO,929,90
1,6997573,407,9045,1,ELMO,929,90
2,6998905,118,9007,0,RUBY,892,100
3,6998905,320,9007,0,RUBY,892,100
4,6998906,219,9008,0,RUBY,892,100


In [10]:
trips_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42864 entries, 0 to 42863
Data columns (total 7 columns):
tripID         42864 non-null int32
vehicle_id     42864 non-null int64
blockID        42864 non-null int64
direction      42864 non-null int64
garage         42864 non-null object
messageCode    42864 non-null int64
routeNumber    42864 non-null int64
dtypes: int32(1), int64(5), object(1)
memory usage: 2.1+ MB


In [11]:
# Now remove these columns except tripID from the event table
events_month.drop(['blockID', 'direction', 'garage', 'messageCode', 'routeNumber'], inplace=True, axis=1)
events_month.head()

,bearing,delay,event_timestamp,lastLocID,lastStopSeq,nextLocID,nextStopSeq,tripID,vehicle_id,vehicle_location_latitude,vehicle_location_longitude
0,147,-31,2017-01-01 08:00:00,13715,13,13716,14,7004120,317,45.483499,-122.643220
1,20,-113,2017-01-01 08:00:04,13720,1,13721,2,7003700,413,45.439829,-122.640634
2,91,-17,2017-01-01 08:00:22,9848,1,9846,2,6998908,110,45.521351,-122.986448
3,358,-56,2017-01-01 08:00:22,9299,6,7763,7,7000472,420,45.526400,-122.676568
4,358,-56,2017-01-01 08:00:22,9299,6,7763,7,7000472,412,45.526400,-122.676568


In [12]:
events_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4601773 entries, 0 to 4601772
Data columns (total 11 columns):
bearing                       int64
delay                         int64
event_timestamp               datetime64[ns]
lastLocID                     int64
lastStopSeq                   int64
nextLocID                     int64
nextStopSeq                   int64
tripID                        int32
vehicle_id                    int64
vehicle_location_latitude     float64
vehicle_location_longitude    float64
dtypes: datetime64[ns](1), float64(2), int32(1), int64(7)
memory usage: 368.6 MB


The main table, "events", is now down to only 8 columns. But with 4.6 million rows, it still takes up quite a bit of memory at 369 MB. Since we're just about down to the lowest number of columns we want to work with, we'll have to sample the number of events (rows). We can do this by using an idea suggest by Jonathan Mackrory. The idea is to only take data entries at each stop, throwing away all the entries where a vehicle is moving in between stops. We can do this quite easily because the delay values for each row are cumulative so there is no need to sum up all entries leading up to each stop. We simply take the last delay value for a particular "lastLocID" and drop the rest.

In [13]:
events_month = events_month.sort_values('event_timestamp')
events_month = events_month.drop_duplicates(['tripID', 'lastLocID'], keep='last')
events_month = events_month.reset_index(drop=True)
events_month.head()

,bearing,delay,event_timestamp,lastLocID,lastStopSeq,nextLocID,nextStopSeq,tripID,vehicle_id,vehicle_location_latitude,vehicle_location_longitude
0,306,-430,2017-01-01 08:00:03,9819,24,9827,25,6999629,251,45.502307,-122.836712
1,20,-113,2017-01-01 08:00:04,13720,1,13721,2,7003700,405,45.439829,-122.640634
2,91,-17,2017-01-01 08:00:22,9848,1,9846,2,6998908,306,45.521351,-122.986448
3,358,-56,2017-01-01 08:00:23,9299,6,7763,7,7000472,420,45.526400,-122.676568
4,98,-729,2017-01-01 08:00:23,8357,46,8359,47,6998905,118,45.501997,-122.419650


In [14]:
events_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164441 entries, 0 to 164440
Data columns (total 11 columns):
bearing                       164441 non-null int64
delay                         164441 non-null int64
event_timestamp               164441 non-null datetime64[ns]
lastLocID                     164441 non-null int64
lastStopSeq                   164441 non-null int64
nextLocID                     164441 non-null int64
nextStopSeq                   164441 non-null int64
tripID                        164441 non-null int32
vehicle_id                    164441 non-null int64
vehicle_location_latitude     164441 non-null float64
vehicle_location_longitude    164441 non-null float64
dtypes: datetime64[ns](1), float64(2), int32(1), int64(7)
memory usage: 13.2 MB


This gets us down to 164,441 rows and a 13.2MB file size. Now a year's worth of data should only be around 160MB. Similar methods are used to generate data for one hour, day, week, and year 